This notebook is a tester notebook for beginning to train and test the data

In [ ]:
#Import the required packages
import scipy
#Import package pandas for data analysis
import pandas as pd

#Import package numpy for numeric computing
import numpy as np

#Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt

#Import package seaborn for visualisation
import seaborn as sns

#For showing plots directly in the notebook run the command below
%matplotlib inline

from matplotlib.backends.backend_pdf import PdfPages

import sqlalchemy
from sqlalchemy import create_engine
from pandas import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score
from sklearn import tree
import matplotlib.pyplot as plt
import pickle
import requests
import json
import numpy as np
import datetime
import time
from time import gmtime
from time import strftime


import warnings
warnings.filterwarnings('ignore')

## Training and testing

In [ ]:
WeatherCombined_df = pd.read_csv("Weather_16_1_Combined.csv")

In [ ]:
y=WeatherCombined_df["JourneyTime"]
X=WeatherCombined_df.drop(["JourneyTime","Holiday", "index", "DayOfService",'dewpt','vappr','rhum','msl','wddir','ww','w','sun','clht', 'ProgrNumber','PROGRNUMBER','TRIPID', 'TripID', 'LineID', 'weather_date','priorstops_actualTime_Arr','PLANNEDTIME_ARR', 'PLANNEDTIME_DEP','ACTUALTIME_ARR','ACTUALTIME_DEP', 'VEHICLEID', 'DAYOFSERVICE', 'RouteID', 'School', 'RushHour', 'Weekend', 'Direction'],axis=1)

In [ ]:
WeatherCombined_df.dtypes

In [ ]:
# WeatherCombined_df['RushHour'] = WeatherCombined_df['RushHour'].astype('datetime64[ns]')

In [ ]:
WeatherCombined_df = WeatherCombined_df[(WeatherCombined_df['JourneyTime'] > 0)]

In [ ]:
object_columns = X.select_dtypes(['object']).columns

In [ ]:
X['STOPPOINTID'] = X['STOPPOINTID'].astype('int')

In [ ]:
for column in object_columns:
    X[column]=X[column].astype('int')

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1)

In [ ]:
X.dtypes

In [ ]:
randForestModel = RandomForestRegressor(n_estimators = 10, random_state=10)
randForestModel.fit(X_train, y_train.values.ravel())

predictionTest = randForestModel.predict(X_test)
forestScore = r2_score(y_test, predictionTest)
forestScore

In [ ]:
decisionTree = DecisionTreeRegressor()
decisionTree.fit(X_train, y_train)
predictions = decisionTree.predict(X_test)
r2_score(y_test, predictions)

In [ ]:
knn_model = KNeighborsRegressor(n_neighbors = 10)
knn_model.fit(X_train, y_train.values.ravel())
knnTest = knn_model.predict(X_test)
knnScore = r2_score(y_test, knnTest)
knnScore

In [ ]:
linear_test = linearModel.predict(X_test)
linear_score = r2_score(y_test, linear_test)
linear_score

In [ ]:
forestPrediction = randForestModel.predict([[1631, 12, 9, 16, 0.2, 9.5, 7.8, 13, 25000, 7]])
forestPrediction

In [ ]:
randForestModel = RandomForestRegressor(n_estimators = 10, random_state=10)
randForestModel.fit(X_train, y_train.values.ravel())
pickle.dump(randForestModel, open(f'randForest_16_1.pkl','wb'))

In [ ]:
from sklearn import metrics

In [ ]:
MAE_test =  metrics.mean_absolute_error(y_test, predictionTest)
MSE_test =  metrics.mean_squared_error(y_test, predictionTest)
RMSE_test = np.sqrt(metrics.mean_squared_error(y_test, predictionTest))

In [ ]:
MAE_test

In [ ]:
prediction = decisionTree.predict([[7349, 12, 9, 16, 0.2, 9.5, 7.8, 13, 25000, 7]])
prediction_values = prediction[0]
print(f"Predicted journey available: {prediction_values}")

In [ ]:
feature_list = X
feature_imp = pd.Series(randForestModel.feature_importances_, index=feature_list)
feature_imp.sort_values(ascending=False)

In [ ]:
linearModel = LinearRegression()
linearModel.fit(X_train, y_train)
linear_prediction = linearModel.predict([[1632, 12, 9, 16, 0.2, 9.5, 7.8, 13, 25000, 7]])
linear_prediction

In [ ]:
numeric=X_train.columns[(X_train.dtypes=="int64") | (X_train.dtypes=="float64")]

In [ ]:
corr=pd.concat([X_train[numeric], y], axis=1).corr()

In [ ]:
plt.figure(figsize=(16,9))
plt.title("Pearson's Correlation of numerical Features for Route 16, direction 1")
sns.heatmap(corr,xticklabels=corr.columns,yticklabels=corr.columns,linewidth=.5,cmap="YlGnBu")
plt.savefig("PearsonsCorr")

In [ ]:
linearModel = LinearRegression()
linearModel.fit(X_train, y_train)
linear_prediction = linearModel.predict([[1127, 12, 9, 16, 0.2, 9.5, 7.8, 13, 25000, 7]])
linear_prediction

In [ ]:
X.columns

In [ ]:
feature_list = X.columns
feature_imp = pd.Series(randForestModel.feature_importances_, index=feature_list)
feature_imp.sort_values(ascending=False)

In [ ]:
def getPrediction(route, direction, stop1, stop2):
    df = pd.read_csv('df_'+ str(route) + '_'+ str(direction)+'.csv')
    #get all stops in the various routes but have them in order
    stops = df['STOPPOINTID']
    indices=[]
    output = []
    stopIDs = [stop1,stop2]
    allStops = []
    total = 0
    
    #get all bus stops in route 16, direction 1
    for stop in stops:
        allStops.append(stop)
     
   #get unique bus stops so we don't get duplicate stops, impportant to keep them in order  
    for x in allStops:
        if x not in output:
            output.append(x)
    
    #get index of chosen bus stops in list of bus stops
    for stop in stopIDs:
        index_number = output.index(stop)
        index_number += 1
        indices.append(index_number)
#     print(indices)
    #use slicing to get the bus stops on the chosen journey
    slice = output[indices[0]:indices[1]]
#     print(slice)
    
    #get predicted journey time of all stops and add them together to get overall journey time
    for stop in slice:
        forest_prediction = pickle.load(open(f'randForest_'+str(route)+'_'+str(direction)+'.pkl', 'rb'))
        prediction = forest_prediction.predict([[stop, 12, 9, 16, 0.2, 9.5, 7.8, 13, 25000, 7]])
        total += forestPrediction 
    print("Prediction in seconds:",total)
        
    

getPrediction(16,1,3669,1632)